In [ ]:
!wget https://raw.githubusercontent.com/IIC2115/Syllabus/main/Actividades%20Pr%C3%A1cticas/A3/catalogo.txt
!wget https://raw.githubusercontent.com/IIC2115/Syllabus/main/Actividades%20Pr%C3%A1cticas/A3/partes.txt
!wget https://raw.githubusercontent.com/IIC2115/Syllabus/main/Actividades%20Pr%C3%A1cticas/A3/proveedores.txt

In [ ]:
import sqlite3
import csv

## Parte 1

Una dirección se puede guardar como un solo string, o separala en más atributos por las comas interiores 
(max 3 y no debe incluir los "")

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS Proveedores(pid INTEGER PRIMARY KEY, nombre TEXT, direccion Text)")
cursor.execute("CREATE TABLE IF NOT EXISTS Partes(paid INTEGER PRIMARY KEY, nombre INTEGER, color TEXT)")
cursor.execute("""CREATE TABLE IF NOT EXISTS Catalogo(pid INTEGER, paid INTEGER, valor REAL, 
FOREIGN KEY (pid) REFERENCES Proveedores, FOREIGN KEY (paid) REFERENCES Partes, PRIMARY KEY(pid, paid))""")
connection.commit()
connection.close()

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

with open('proveedores.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO Proveedores VALUES (?,?,?)', line)

with open('partes.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO Partes VALUES (?,?,?)', line)
        
with open('catalogo.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO Catalogo VALUES (?,?,?)', line)
        
connection.commit()
connection.close()

NameError: ignored

## Parte 2 Consultas

Supuesto de que los nombres de los proveedores son unicos. En caso contrario, los distinct se deberian usar en los ids.

Consulta 1

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("""SELECT DISTINCT Proveedores.nombre FROM Proveedores, Partes, Catalogo
                  WHERE Proveedores.pid = Catalogo.pid AND Partes.paid = Catalogo.paid AND Partes.color = 'Red' """)

for parte in cursor.fetchall():
    print(parte[0])
    
connection.commit()
connection.close()

Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts


Consulta 2

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

cursor.execute("""SELECT Partes.nombre, Partes.color FROM Catalogo, Proveedores, Partes
                 WHERE Partes.paid= Catalogo.paid AND Catalogo.pid=Proveedores.pid
                 AND Proveedores.nombre LIKE '%ACME%' """)

for parte in cursor.fetchall():
    print(parte[0])
    
connection.commit()
connection.close()

Acme Widget Washer
Acme Widget Washer
Fire Hydrant Cap


Consulta 3

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

cursor.execute("""SELECT Distinct(Proveedores.nombre) FROM Catalogo, Proveedores, Partes,
                 (SELECT Partes.paid, AVG(Catalogo.valor) as promedio FROM Catalogo, Partes
                 WHERE Partes.paid= Catalogo.paid
                 GROUP BY Catalogo.paid) as Tablapromedios
                 WHERE Partes.paid=Catalogo.paid AND Catalogo.pid= Proveedores.pid
                 AND Tablapromedios.paid= Partes.paid AND Catalogo.valor> Tablapromedios.promedio
                 """)

for parte in cursor.fetchall():
    print(parte[0])
    
connection.commit()
connection.close()

Big Red Tool and Die
Acme Widget Suppliers
Perfunctory Parts


Consulta 4

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

cursor.execute("""SELECT Partes.nombre, MAX(Catalogo.valor) FROM Partes, Catalogo, Proveedores
                 WHERE Partes.paid= Catalogo.paid AND Catalogo.pid=Proveedores.pid
                 AND Proveedores.pid IN (SELECT Proveedores.pid FROM Partes, Catalogo, Proveedores
                                         WHERE Partes.paid= Catalogo.paid AND Catalogo.pid=Proveedores.pid AND Partes.color='Green'
                                         AND Proveedores.pid IN (SELECT Proveedores.pid FROM Partes, Catalogo, Proveedores
                                                                 WHERE Partes.paid= Catalogo.paid AND Catalogo.pid=Proveedores.pid
                                                                 AND Partes.color='Red'))
                GROUP BY Proveedores.pid""")

for parte in cursor.fetchall():
    print(parte)
    
connection.commit()
connection.close()

('Fire Hydrant Cap', 12.5)
